In [1]:
# Import required Python Libraries
import numpy as np
import math
import itertools
import random

### 1. get_ground_state( )
This function accepts the total number of qubits and returns the Ground State.<br>
For 2 qubits, the ground state will be [1,0,0,0]. It represents 100% probability for 00 and 0% probability for 01,10, and 11.<br>
For 3 qubits, the ground state will be [1,0,0,0,0,0,0,0].It represents 100% probability for 000 and 0% probability for 001,010,100,011,110,101 and 111.

In [2]:
def get_ground_state(num_qubits):
    a = 2**num_qubits -1
    b = [1]
    for i in range(a):
        b.append(0)
    return b

### 2. get_operator( )
This function accepts the total number of qubits, gate, and the target qubit on which the Gate needs to be applied. <br>
It returns the Matrix Operator for the Gate (programmed to work for X,H, and CNOT Gates only)<br>
Matrix Operator is computed by calculating the Tensor Product of Gate Matrix and 2X2 Identity Matrix<br>
For example, for single-qubit gate U and CNOT Gate in 3-qubit circuit:<br>
Operator for U Gate on qubit 0 = U (X) I (X) I (where (X) is Tensor Product)<br>
Operator for U Gate on qubit 1 = I (X) U (X) I <br>
Operator for U Gate on qubit 2 = I (X) I (X) U <br>
Operator for CNOT Gate on qubit 0 and 1 = CNOT (X) I <br>
Operator for CNOT Gate on qubit 1 and 2 = I (X) CNOT

In [3]:
def get_operator(total_qubits, gate_unitary, target_qubits):

    I = np.identity(2)
 
    if gate_unitary == 'x':
        X = np.array([[0, 1],
                      [1, 0]])
        
        result = None
        for i in range(total_qubits):
            operand = (X if i == target_qubits[0] else I)
            if result is None:
                result = operand
            else:
                result = np.kron(result,operand)
        return result
            
    if gate_unitary == 'h':
        H = np.array([
            [1/np.sqrt(2), 1/np.sqrt(2)],
            [1/np.sqrt(2), -1/np.sqrt(2)]
            ])
        
        result = None
        for i in range(total_qubits):
            operand = (H if i == target_qubits[0] else I)
            if result is None:
                result = operand
            else:
                result = np.kron(result,operand)
        return result      
        
        
    if gate_unitary == 'cx':
        CNOT = np.array([[1, 0, 0, 0],
                         [0, 1, 0, 0],
                         [0, 0, 0, 1],
                         [0, 0, 1, 0]])
        
        result = None
        for i in range(total_qubits):
            if i==0:
                continue
            operand = (CNOT if i == target_qubits[1] else I)
            if result is None:
                result = operand
            else:
                result = np.kron(result,operand)
        return result  


### 3. run_program( )
This function accepts the initial state and the Quantum Circuit in the form of Gate and the qubit on which the Gate needs to be applied.<br>
It returns the Final State after the Gate has been applied.

In [4]:
def run_program(initial_state, program):
    total_qubits = int(math.log(len(initial_state),2))
    
    psi = initial_state
    for i in range(len(program)): 
        gate_unitary = program[i]["gate"]
        target_qubits = program[i]["target"]
        op = get_operator(total_qubits,gate_unitary,target_qubits)
        psi = np.dot(op,psi)
    return psi

### 4. get_counts( )
The Final State and the number of shots are passed to this function. <br>
It calculates the probabilities fron the state vector and returns the random counts based on the weighted probabilities.

In [5]:
def get_counts(state_vector, num_shots):
    total_qubits = int(math.log(len(state_vector),2))
    pop= ["".join(seq) for seq in itertools.product("01", repeat=total_qubits)]
    prob= state_vector**2
    a=random.choices(pop,final_state**2,k=num_shots)
    
    output = dict()
    for i in range(len(pop)):
        if a.count(pop[i]) == 0:
            continue
        output[pop[i]] = a.count(pop[i])
    return output
    

## Run the Program
1. Provide the Gate and the Qubit on which the Gate needs to be applied. <br>
2. Provide the total number of Qubits on which the program needs to be based. <br>
3. Provide the number of shots. <br>

Result: The Program returns the counts of Basis States for the number of shots provided.

In [6]:
my_circuit = [
  { "gate": "h", "target": [0] },
  { "gate": "x", "target": [1] },
  { "gate": "cx", "target": [1,2] }
]

my_qpu=get_ground_state(3)

final_state=run_program(my_qpu, my_circuit)

counts = get_counts(final_state, 1000)

for key, value in counts.items():
    print(key, ' : ', value)

011  :  494
111  :  506
